<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/117_IO%E3%82%B9%E3%83%88%E3%83%AA%E3%83%BC%E3%83%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I/O ストリーム
==============

I/O ストリーム
--------------

連続したデータを「流れるもの」として捉え、ファイルやメモリなどからデータを読み取ったり書き込んだりする際のデータの流れを**入出力ストリーム**または **I/O ストリーム**と呼んでいる。標準ライブラリの `io` モジュールは、I/O ストリームを扱うためのインターフェースを次のように構成している。

| 型 | 意味 | 抽象基底クラス | 継承元 | スタブメソッド | Mixin するメソッドとプロパティ |
|:---|:---|:---|:---|:---|:---|
| IOBase | 読み込み、書き込み、シークができないストリーム | `IOBase` | | `fileno`, `seek`, `truncate` | `closed`（プロパティ）,<br />`close`, `flush`, `isatty`, `readable`, `readline`, `readlines`,<br />`seekable`, `tell`, `writable`, `writelines`,<br />`__enter__`, `__exit__`, `__iter__`, `__next__` |
| Raw I/O | OS に近い部分とやり取りするストリーム | `RawIOBase` | `IOBase` | `readinto`, `write` | `IOBase` から継承したメソッド、`read`, `readall` |
| バイナリ I/O | バッファを持つストリーム | `BufferedIOBase` | `IOBase` | `detach`, `read`, `read1`, `write` | `IOBase` から継承したメソッド、`readinto`, `readinto1` |
| テキスト I/O | テキストとバイナリの変換機構を持つストリーム | `TextIOBase` | `IOBase` | `detach`, `read`, `readline`, `write` | `encoding`, `errors`, `newlines`（プロパティ）,<br />`IOBase` から継承したメソッド |

I/O ストリームは、次の基本的な性質を持つ。

  1. I/O ストリームはイテレータープロトコルをサポートする。オブジェクトをイテレートすると、ストリーム内の現在位置以降の行が yield される。ストリーム内の行の定義は、そのストリームが（バイト列を yield する）バイナリストリームか（文字列を yield する）テキストストリームかによって、少し異なる。現在位置より前にある行は yield されないので、それらを yield したいなら、イテレートを開始する前にストリーム内の位置をそれらの行より前に変更しておく必要がある。
  2. I/O ストリームはコンテキストマネージャーでもある。そのため with 構文をサポートする。`__enter__()` メソッドはオブジェクト自身を返す。`__exit__()` メソッドはオブジェクトの `close()` メソッドを実行する。

Raw I/O、バイナリ I/O、テキスト I/O のいずれかの具象オブジェクトは全て**ファイルオブジェクト**（または **file-like オブジェクト**）と呼ばれる。ファイルオブジェクトは、読み込み専用、書き込み専用、読み書き可能のいずかになる。

`io` モジュールは、Raw I/O、バイナリ I/O、テキスト I/O の各カテゴリーに属する I/O ストリームの実装（具象クラス）を提供している。ただし、それらは Python コードによるクラスではなく C 実装となっている。クラス階層については、「朝日ネット　技術者ブログ」の記事にある[図表](https://techblog.asahi-net.co.jp/entry/2021/10/04/162109#io-%E3%83%A2%E3%82%B8%E3%83%A5%E3%83%BC%E3%83%AB%E3%81%AB%E3%81%A4%E3%81%84%E3%81%A6%E7%9F%A5%E3%82%8B)を参照。

FileIO
------

クラス `io.FileIO` は、OS レベルのファイルを表現するバイナリストリームの実装である。`io.RawIOBase` を継承している。

コンストラクタ:

``` python
io.FileIO(name, mode='r', closefd=True, opener=None)
```

`name` 引数に渡せるのは、次の 2 つのいずれかである。

  * 開くファイルへのパスを表す文字列またはバイト列。この場合、`closefd` は `True`（デフォルト）でなければならない。
  * ファイルディスクリプタ（fd）。ファイルディスクリプタとは、OS にアクセスを依頼する際にファイルを指定するのに用いられる整数値である。通常、ファイルディスクリプタは 0 から順番に未使用の最も小さい値が与えられるようになっている。とくに、0 は標準入力、1 は標準出力、2 は標準エラー出力に固定される。`FileIO` オブジェクトが閉じられると、`closefd` が `False` に設定されていない場合、この fd も閉じられる。

`mode` 引数については次のとおり:

| mode | 読み込み | 書き込み | ファイルが存在しない場合 | ファイルが存在する場合 |
|:--:|:--:|:--:|:---|:---|
| `'r'` | 〇 | × | エラー（FileNotFoundError） | 既存の内容は維持される |
| `'w'` | × | 〇（切り詰め） | 新規作成 | 既存の内容は<font color="red">削除</font>される |
| `'x'` | × | 〇 | 新規作成 | エラー（FileNotFoundError） |
| `'a'` | × | 〇（末尾に追記） | 新規作成 | 既存の内容は維持される |
| `'r+'` | 〇 | 〇（先頭から上書き） | エラー（FileNotFoundError） | 既存の内容は維持される |
| `'w+'` | 〇 | 〇（切り詰め） | 新規作成 | 既存の内容は<font color="red">削除</font>される |
| `'a+'` | 〇 | 〇（末尾に追記） | 新規作成 | 既存の内容は維持される |

新規作成の場合、 `name` 引数に指定したパスでファイルの直上までのディレクトリが存在していないとエラー（`FileNotFoundError`）となる。

`io.FileIO()` コンストラクタは、ファイルを開くシステムコールを実行するが、呼び出し可能オブジェクトを `opener` 引数として渡すと、この機能をカスタマイズできる。`os.open` を `opener` として渡すと、`None` を渡したのと同様の機能になる。

`io.FileIO` のプロパティ（読み出し専用）:

| プロパティ | 意味 |
|:---|:---|
| `mode` | コンストラクタに渡された同名の引数の値 |
| `name` | ファイル名。コンストラクタに名前が渡されなかったときはファイルディスクリプタになる |
| `closed` | ストリームが閉じられていた場合 True になる |

`io.FileIO` のメソッド:

| メソッド | 機能 | 戻り値 |
|:---|:---|:--:|
| `fileno()` | コンストラクタにファイルディスクリプタが渡されていた場合はそれを返す。そうでなければ、`OSError` 例外が発生する | `int` |
| `isatty()` | ストリームが対話的であれば（つまりターミナルや tty デバイスにつながっている場合）`True` を返す | `bool` |
| `readable()` | ファイルが読み込み可能で開かれている場合、`True` を返す。`False` の場合、`read*()` を使用すると `OSError` 例外が発生する | `bool` |
| `readline(size=-1, /)` | ファイルから 1 行読み込んで返す。`size` が指定された場合、最大で `size` バイトが読み込まれる | `bytes` |
| `readlines(hint=-1, /)` | ファイルから行のリストを読み込んで返す。読み込んだすべての行のサイズ（バイト数）が `hint` の値を超えた場合、それ以上の行は読み込<br />まれない。引数 `hint` の値が 0 以下、および `None` の場合は、`hint` による制限はない<br />※ 反復処理には、この `readlines()` メソッドを使うのではなく、ファイルオブジェクトが 1 行ずつ返すイテレーターであることを利用すべき | `list` |
| `read(size=-1, /)` | ファイルを `size` バイトまで読み込み、それを返す。`size` が `-1`（デフォルト）の場合は、EOF までの全てのバイトを返す。すでに EOF に到達<br />していて 1 バイトも読み込めなければ、`b''` が返される | `bytes` |
| `readall()` | EOF までファイルから全てのバイトを読み込み、それを返す | `bytes` |
| `readinto(b, /)` | 事前に定義したフォーマット `b` にバイト列を代入し、読み込んだバイト数を返す | `bytes` |
| `writable()` | ファイルが書き込み可能で開かれている場合、`True` を返す。`False` の場合、`write*()` を使用すると `OSError` 例外が発生する | `bool` |
| `writelines(lines, /)` | ファイルにリスト `lines` を書き込む | `None` |
| `write(b, /)` | バイト列 `b` をファイルに書き込み、書き込んだバイト数を返す | `int` |
| `seekable()` | ストリームがランダムアクセスをサポートしている場合、`True` を返す。`False` の場合、`tell()`、`seek()`、`truncate()` を使用すると `OSError` <br />例外が発生する | `bool` |
| `tell()` | 現在のストリーム位置を返す | `int` |
| `seek(offset, whence=os.SEEK_SET, /)` | ストリーム位置を、`whence` で示される基準点にオフセット値 `offset` を足した位置に変更し、変更後の位置（先頭からの位置）を返す。<br />`whence` の値は以下の通り<br /><br />・`os.SEEK_SET` または `0`: 基準点はストリームの開始位置（デフォルト）。`offset` は 0 または正である必要がある<br /><br />・`os.SEEK_CUR` または `1`: 基準点はストリームの現在位置。オフセットは負の値でもよい<br /><br />・`os.SEEK_END` または `2`: 基準点はストリームの末尾。オフセットは通常は負の値である | `int` |
| `truncate(size=None, /)` | ストリームのサイズを、指定された `size` バイト（または `size` が指定されていない場合、現在位置）に変更する。現在のストリーム位置は変<br />更されない。このサイズ変更により、現在のファイルサイズを拡大または縮小させることができる。拡大の場合には、新しいファイル領域の<br />内容は `b' '` で埋められる。新しいファイルサイズが返される | `int` |
| `flush()` | 何も起こらない | `None` |
| `close()` | ファイルを閉じる。いったんファイルが閉じられると、全てのファイルに対する操作（読み込みや書き込みなど）で `ValueError` 例外が発生<br />する | `None` |

ある種のバイナリファイルは、フォーマット仕様が規定される。たとえば、WAV ファイルに採用される [RIFF 形式](https://ja.wikipedia.org/wiki/Resource_Interchange_File_Format)は、「チャンク」と呼ばれるものの並びであり、全てのチャンクは次のような形式である。

  * 4 バイト: チャンクの種類を表現するASCII文字列（チャンク識別子と呼ばれる）。たとえば `b'fmt'`、`b'data'` など
  * 4 バイト: データサイズを示す 32 ビットの符号なし整数
  * 可変長フィールド: データの本体

データの本体の中にチャンクを入れ子にすることができ、これをサブチャンクと呼ぶ。

WAV ファイルのヘッダーとなるチャンクには、識別子として `b'RIFF'` が指定され、データの本体には、最初の 4 バイトに `b'WAVE'` が指定され、それ以降はサブチャンクが連なっている。

データフォーマットを定義するには、標準ライブラリの `ctypes` モジュールが提供する `ctypes.Structure` クラスを継承したクラスを作成する。このクラスでは、`_fields_` 属性を定義する。そこには、フィールドの名前とフィールドの型をセットにしたタプルを記述していく。次のコードは、データフォーマットを定義して、`io.FileIO()` を使って WAV ファイルのヘッダーとフォーマットを読み込んでいる。

``` python
import ctypes as c
import io

# フォーマットを定義
class WAVHeaderFormat(c.Structure):
    _fields_ = [
        ("id", c.c_char * 4),  # 4バイトの文字型
        ("size", c.c_uint32),  # 符号なし32ビット整数型
        ("type", c.c_char * 4),  # 4バイトの文字型
    ]

with io.FileIO("./sample.wav", "r") as f:
    format_data = WAVHeaderFormat()
    i = f.readinto(format_data)  # format_data にデータを入れ込む
    print(f"{i=}")  # 4 + 4 + 4 = 12 バイト読み込んだので 12 が出力される
    print(f"{format_data.id=}")  # b'RIFF' が出力される
    print(f"{format_data.size=}")  # WAV ファイルのデータサイズが出力される
    print(f"{format_data.type=}")  # b'WAVE' が出力される
```

Wikimedia Commons の[音声データサイト](https://commons.wikimedia.org/wiki/File:342706_spacejoe_lock-3-open-lock-3.wav)から WAV ファイルをウンロードし、ファイル名を `sample.wav` に変更してから上のコードを実行した結果、次のように出力された。

``` text
i=12
format_data.id=b'RIFF'  
format_data.size=94288  
format_data.type=b'WAVE'
```

ネイティブではないバイトオーダーのデータフォーマットを定義するには、`ctypes.BigEndianStructure` または `ctypes.LittleEndianStructure` を継承したクラスを作成する。詳しくは、[公式ドキュメント](https://docs.python.org/ja/3/library/ctypes.html)を参照。

BytesIO
-------

クラス `io.BytesIO` は、メモリ上のバイトバッファを利用したバイナリストリームの実装である。`io.BufferedIOBase` を継承している。

コンストラクタ:

``` python
io.BytesIO(initial_bytes=b'')
```

`initial_bytes` 引数には初期値となるバイト列を指定できる。

`io.BytesIO` のプロパティ（読み出し専用）:

| プロパティ | 意味 |
|:---|:---|
| `closed` | ストリームが閉じられていた場合 True になる |

`io.BytesIO` のメソッド:

| メソッド | 機能 | 戻り値 |
|:---|:---|:--:|
| `detach()` | `io.BytesIO` ではサポートされない（`io.UnsupportedOperation` 例外を送出する） | |
| `getbuffer()` | バッファの内容を、読み込み及び書き込みが可能なビューとして返す。このビューを更新すると、バッファの内容も更新される | `MemoryView` |
| `getvalue()` | バッファの内容を返す。この値は更新できない | `bytes` |
| `readable()` | `io.BytesIO` においては、このメソッドは常に `True` を返す | `bool` |
| `readline(size=-1, /)` | バッファから 1 行読み込んで返す。`size` が指定された場合、最大で `size` バイトが読み込まれる。行末文字は常に `b'\n'` となる | `bytes` |
| `readlines(hint=-1, /)` | バッファから行のリストを読み込んで返す。読み込んだすべての行のサイズ（バイト数）が `hint` の値を超えた場合、それ以上の行は読み込ま<br />れない。引数 `hint` の値が 0 以下、および `None` の場合は、`hint` による制限はない<br />※ 反復処理には、この `readlines()` メソッドを使うのではなく、ファイルオブジェクトが 1 行ずつ返すイテレーターであることを利用すべき | `list` |
| `read(size=-1, /)` | バッファを `size` バイトまで読み込み、それを返す。`size` が `-1`（デフォルト）の場合は、EOF までの全てのバイトを返す。すでに EOF に到達し<br />ていて 1 バイトも読み込めなければ、`b''` が返される | `bytes` |
| `read1(size=-1, /)` | `io.BytesIO` においては、このメソッドは `read()` と同じ | `bytes` |
| `readinto(b, /)` | 事前に定義したフォーマット `b` にバイト列を代入し、読み込んだバイト数を返す | `bytes` |
| `readinto1(b, /)` | `io.BytesIO` においては、このメソッドは `readinto()` と同じ | `bytes` |
| `writable()` | `io.BytesIO` においては、このメソッドは常に `True` を返す | `bool` |
| `writelines(lines, /)` | バッファにリスト `lines` を書き込む | `None` |
| `write(b, /)` | バイト列 `b` をバッファに書き込み、書き込んだバイト数を返す | `int` |
| `seekable()` | `io.BytesIO` においては、このメソッドは常に `True` を返す | `bool` |
| `tell()` | 現在のストリーム位置を返す | `int` |
| `seek(offset,`<br />` whence=os.SEEK_SET, /)` | ストリーム位置を、`whence` で示される基準点にオフセット値 `offset` を足した位置に変更し、変更後の位置（先頭からの位置）を返す。`whence` <br />の値は以下の通り<br /><br />・`os.SEEK_SET` または `0`: 基準点はストリームの開始位置（デフォルト）。`offset` は 0 または正である必要がある<br /><br />・`os.SEEK_CUR` または `1`: 基準点はストリームの現在位置。オフセットは負の値でもよい<br /><br />・`os.SEEK_END` または `2`: 基準点はストリームの末尾。オフセットは通常は負の値である | `int` |
| `truncate(size=None, /)` | ストリームのサイズを、指定された `size` バイト（または `size` が指定されていない場合、現在位置）に変更する。現在のストリーム位置は変更<br />されない。このサイズ変更により、現在のバッファサイズを変更できる（拡大はできない？）。新しいバッファサイズが返される | `int` |
| `flush()` | 何も起こらない | `None` |
| `close()` | バイトバッファを破棄する。`close()` の後にバッファに対する操作（読み込みや書き込みなど）で `ValueError` 例外が発生する | `None` |

In [ ]:
import io

with io.BytesIO(b"abcdef") as stream:
    assert stream.tell() == 0  # 現在位置（最初は 0）
    assert stream.read(2) == b"ab"  # 指定サイズだけ読みだす
    assert stream.seek(0, io.SEEK_END) == 6  # ストリーム位置を末尾に変更
    stream.write(b"test")  # ストリームに文字列を書き込む
    assert stream.getvalue() == b"abcdeftest"

b = io.BytesIO(b"abcdef")
view = b.getbuffer()  # 読み込みおよび書き込みが可能なビューを取得
view[2:4] = b"56"
assert b.getvalue() == b"ab56ef"  # ビューの更新結果が反映される

BufferedReader・BufferedWriter・BufferedRandom
----------------------------------------------

クラス `io.BufferedReader` および `io.BufferedWriter`、`io.BufferedRandom` は、クラス `io.FileIO` に対して修正を加えることなく、インタフェースを変更するラッパークラスであり、バッファされたバイナリストリームの実装である。`io.BufferedReader` および `io.BufferedWriter` は、`io.BufferedIOBase` を継承している。`io.BufferedRandom` は、`io.BufferedReader` と `io.BufferedWriter` を継承している。

いずれもコンストラクタの引数は同じなので、`io.BufferedReader` のコンストラクタのみ示す:

``` python
io.BufferedReader(raw, buffer_size=io.DEFAULT_BUFFER_SIZE)
```

| 引数 | 意味 |
|:---|:---|
| `raw` | `io.FileIO` のインスタンスであるバイナリストリーム |
| `buffer_size` | バッファのサイズ。デフォルト値は `io.DEFAULT_BUFFER_SIZE` |

In [ ]:
import io
io.DEFAULT_BUFFER_SIZE

8192

3 つのクラスは共通して次の読み出し専用プロパティを持つ:

| プロパティ | 意味 |
|:---|:---|
| `raw` | コンストラクタの同名の引数に渡されたバイナリストリーム |

また、`raw` に格納されたストリームのプロパティ（`closed`、`mode`、`name`）を自身のプロパティとして保持する。

`io.BufferedReader` と `io.BufferedWriter` は、`io.FileIO` のメソッドに加えて、以下のメソッドを提供または拡張する:

| メソッド | 機能 | 戻り値 |
|:---|:---|:--:|
| `detach()` | `raw` に格納されたストリームをバッファから分離して返す。ストリームが取り外された後、バッファは使用不能状態になる | `RawIOBase` |
| `readable()` | `raw` が読み込み可能な場合、`True` を返す。そうでないとき `io.BufferedReader` においては `io.UnsupportedOperation` 例外を送出する | `bool` |
| `writable()` | `raw` が書き込み可能な場合、`True` を返す。そうでないとき `io.BufferedWriter` においては `io.UnsupportedOperation` 例外を送出する | `bool` |

`io.BufferedReader` は、以下のメソッドを提供または拡張する:

| メソッド | 機能 | 戻り値 |
|:---|:---|:--:|
| `peek(size=0, /)` | 位置を進めずにファイルからバイト列を返す | `bytes` |
| `read(size=-1, /)` | ファイルから `buffer_size` 単位でデータを読み出すことを繰り返し、読み出したバイト数の合計が `size` バイトを超えたら読み出しを終了し、読み出した<br />`size` バイトのデータを返す。読み出したデータはバッファに保持され、その後の読み出し処理で直接返すことができる。`size` が `-1`（デフォルト）の場合<br />は、EOF までの全てのバイトを返す。すでに EOF に到達していて 1 バイトも読み出せなければ、`b''` が返される | `bytes` |
| `read1(size=-1, /)` | `raw` に格納されたストリームの `read()`（または `readinto()`） メソッドを高々 1 回呼び出し、最大で `size` バイト読み込んで返す。これは、`BufferedIOBase`<br />オブジェクトの上に独自のバッファリングを実装するときに使うことを想定している | `bytes` |

`io.BufferedWriter` は、以下のメソッドを提供または拡張する:

| メソッド | 機能 | 戻り値 |
|:---|:---|:--:|
| `writelines(lines, /)` | バッファにリスト `lines` を書き込む | `None` |
| `write(b, /)` | バイト列 `b` をバッファに書き込み、書き込んだバイト数を返す | `int` |
| `flush()` | バッファに保持されたバイト列を、`raw` に格納されたストリームに強制的に流し込む（結果的にファイルへの書き込みが行われる） | `None` |
| `close()` | `flush()` を実行し、`raw` に格納されたストリームを閉じる。`close()` の後にバッファに対する操作（読み込みや書き込みなど）で `ValueError` 例外が発生する | `None` |

StringIO
--------

`io.StringIO` は、メモリ上のテキストバッファを利用したテキストストリームの実装である。`io.TextIOBase` を継承している。

コンストラクタ:

``` python
`io.StringIO(initial_value='', newline='\n')`
```

| 引数 | 意味 |
|:---|:---|
| `initial_value` | 初期値となる文字列を指定する |
| `newline` | 改行文字をどのようにパースするかを指定する<br /><br />・`None` の場合、入力に含まれる `'\r\n'`, `'\n'`, `'\r'` をすべて `\n` に変換する。出力時に改行を変換しない（`\n` のまま）<br /><br />・`''` を指定した場合、入力の `'\r\n'`, `'\n'`, `'\r'` を行末として扱うが `'\n'` に自動変換はせず、出力時に `'\n'` を自動変換することもない<br /><br />・`'\n'`, `'\r'`, `'\r\n'` のいずれかを指定した場合、入力時に指定の文字列のみを行末として扱い、出力時に `'\n'` を指定の文字に自動変換する |

`io.StringIO` のプロパティ（読み出し専用）:

| プロパティ | 意味 |
|:---|:---|
| `encoding` | `io.StringIO` においては `None` に固定 |
| `errors` | `io.StringIO` においては `None` に固定 |
| `newlines` | `io.StringIO` においては `None` に固定 |
| `closed` | ストリームが閉じられていた場合 `True` になる |

`io.StringIO` のメソッド:

| メソッド | 機能 | 戻り値 |
|:---|:---|:--:|
| `getvalue()` | バッファが保持しているすべての内容を文字列として返す | `str` |
| `readable()` | `io.StringIO` においては 、このメソッドは常に `True` を返す | `bool` |
| `read(size=-1, /)` | 最大 `size` 文字をバッファから読み込み、一つの `str` にして返す。`size` が負の値または `None` ならば、EOF まで読む | `str` |
| `readline(size=-1, /)` | バッファから 1 行読み込んで返す。`size` が指定された場合、最大で `size` 文字が読み込まれる。すでに EOF に達していた場合、空文字列を返す | `str` |
| `readlines(hint=-1, /)` | バッファから行のリストを読み込んで返す。読み込んだすべての行のサイズ（文字数）が `hint` の値を超えた場合、それ以上の行は読み込まれない。引<br />数 `hint` の値が 0 以下、および `None` の場合は、`hint` による制限はない<br />※ 反復処理のためなら、この `readlines()` メソッドを呼び出すのではなく、ファイルオブジェクトが 1 行ずつ返すイテレーターであることを利用すべき | `list` |
| `writable()` | `io.StringIO` においては 、このメソッドは常に `True` を返す | `bool` |
| `write(s, /)` | 文字列 `s` をバッファに書き出し、書き出された文字数を返す | `int` |
| `writelines(lines, /)` | バッファに行のリストを書き込む。行区切り文字は追加されないので、書き込む各行の行末に行区切り文字を含ませるのが一般的である | `None` |
| `seekable()` | `io.StringIO` においては 、このメソッドは常に `True` を返す | `bool` |
| `tell()` | 現在のストリーム位置を返す | `int` |
| `seek(offset, whence=0 /)` | ストリーム位置を、`whence` で示される基準点にオフセット値 `offset` を足した位置に変更し、変更後の位置（先頭からの位置）を返す。`whence` の値は<br />以下の通り<br /><br />・`os.SEEK_SET` または 0: 基準点はストリームの先頭。`offset` には 0 または `tell()` が返す値を指定できる<br /><br />・`os.SEEK_CUR` または 1: 基準点は現在のストリーム位置。`offset` には 0 のみ指定できる（つまり何もしない）<br /><br />・`os.SEEK_END` または 2: 基準点はストリームの末尾。`offset` には 0 のみ指定できる | `int` |
| `truncate(size=None, /)` | バッファのサイズを、指定された `size` バイト（または `size` が指定されていない場合、現在位置）に変更する。現在のストリーム位置は変更されない。<br />このサイズ変更により、現在のバッファサイズを変更できる（拡大はできない？）。新しいバッファサイズが返される | `int` |
| `flush()` | 何も起こらない | `None` |
| `close()` | テキストバッファを破棄する。`close()` の後にバッファに対する操作（読み込みや書き込みなど）で `ValueError` 例外が発生する | `None` |

In [ ]:
import io

with io.StringIO("Spam") as stream:
    assert stream.tell() == 0  # 現在位置（最初は 0）
    assert stream.read(2) == "Sp"  # 指定サイズだけ読みだす
    assert stream.seek(0, io.SEEK_END) == 4  # ストリーム位置を末尾に変更
    stream.write(" and eggs!\n")  # ストリームに文字列を書き込む
    assert stream.getvalue() == "Spam and eggs!\n"
    stream.write("Spam and ham!")  # ストリームに文字列を書き込む
    assert stream.seek(0) == 0  # ストリーム位置を先頭に変更
    for line in stream:
        print(line.rstrip())

print("-" * 40)
assert stream.closed  # with 文を抜けるとき close() が実行される
try:
    stream.write("test")  # 閉じたあとに操作しようとすると、エラーが発生する
except ValueError as err:
    print(f"{type(err).__name__}: {err}")

Spam and eggs!
Spam and ham!
----------------------------------------
ValueError: I/O operation on closed file


TextIOWrapper
-------------

クラス `io.TextIOWrapper` は、クラス `io.BufferedReader` および `io.BufferedWriter`、`io.BufferedRandom` に対して修正を加えることなく、インタフェースを変更するラッパークラスであり、バッファされたテキストストリームの実装である。`io.TextIOBase` を継承している。

コンストラクタ:

``` python
io.TextIOWrapper(buffer, encoding=None, errors=None, newline=None, line_buffering=False, write_through=False)
```

各引数は、それぞれ対応するインスタンス属性の値を指定する。

`io.TextIOWrapper` のプロパティ（読み出し専用）:

| プロパティ | 意味 |
|:---|:---|
| `buffer` | `io.BufferedIOBase` の具象クラスのインスタンスである、バッファ付きバイナリストリーム |
| `encoding` | ファイルのエンコードやデコードに使われるエンコーディングの名前（ `'utf-8'` など）。コンストラクタの引数に `'locale'` または `None` を指定した場合、システムの<br />ロケールの文字コード（`locale.getencoding()` で与えられる文字コード）となる<br />・Windows: ANSI（日本語環境では `'cp932'`）<br />・Unix 系: `'utf-8'` |
| `errors` | エンコードやデコードでのエラーをどのように扱うかを指定する。コンストラクタの引数に `None`（デフォルト）を指定した場合、このプロパティは `'strict'` となる<br /><br />・`'strict'`: エンコーディングエラーがあると `ValueError` 例外を発生させる<br /><br />・`'ignore'`: エラーを無視する<br /><br />・`'replace'`: 不正な形式のデータが存在した場所に（`?` のような）置換マーカーを挿入する |
| `newline` | `io.StringIO` の同名の引数と同じように働く。ただし、`None` の場合、入出力に含まれる改行を `\n` と相互に自動変換する。たとえば Windows では入力に含まれる<br /> `'\r\n'`（CR+LF）は Python 側で `'\n'`（LF）に変換され、出力する際は `'\n'` は `'\r\n'` に変換される |
| `line_buffering` | `True` の場合、書き込み時に `'\n'` や `'\r'` が含まれると、`flush()` が暗黙的に実行され、ファイルへの書き込みが行われる |
| `write_through` | `True` の場合、`write()` の呼び出しで即座にファイルへの書き込みが行われる |

また、`io.BufferedIOBase` の具象クラスのインスタンスのファイルに関するプロパティ（`closed`、`mode`、`name`）を自身のプロパティとして保持する。

`io.TextIOWrapper` のメソッド:

| メソッド | 機能 | 戻り値 |
|:---|:---|:--:|
| `detach()` | `buffer` に格納されたバッファを `io.TextIOWrapper` から分離して返す。その後は`io.TextIOWrapper` は使用不能状態になる | `BufferedIOBase` |
| `reconfigure(*, encoding=None,`<br />` errors=None, newline=None,`<br />` line_buffering=None,`<br />` write_through=None)` | このテキストストリームを、 `encoding`, `errors`, `newline`, `line_buffering`, `write_through` を新しい設定として再設定する。ただし、ストリーム<br />からすでにデータが読み出されていた場合、`encoding` と `newline` は変更できない。一方で、書き込み後に `encoding` を変更することはでき<br />る。指定されなかったパラメータは現在の設定が保持される。このメソッドは、新しい設定を適用するまえにストリームをフラッシュする | `None` |
| `readable()` | `buffer` に格納されたバッファが読み出し可能なときに `True` を返す | `bool` |
| `read(size=-1, /)` | 最大 `size` 文字をストリームから読み込み、一つの `str` にして返す。`size` が負の値または `None` ならば、EOF まで読む | `str` |
| `readline(size=-1, /)` | ストリームから 1 行読み込んで返す。`size` が指定された場合、最大で `size` 文字が読み込まれる。行末文字は常に `'\n'` となる。空行を読<br />み込んだ場合は `'\n'` を返し、すでに EOF に達していた場合は空文字列 `''` を返す | `str` |
| `readlines(hint=-1, /)` | バッファから行のリストを読み込んで返す。読み込んだすべての行のサイズ（文字数）が `hint` の値を超えた場合、それ以上の行は読み込ま<br />れない。引数 `hint` の値が 0 以下、および `None` の場合は、`hint` による制限はない<br />※ 反復処理のためなら、この `readlines()` メソッドを呼び出すのではなく、ファイルオブジェクトが 1 行ずつ返すイテレーターであることを<br />　利用すべき | `list` |
| `writable()` | `buffer` に格納されたバッファが書き込み可能なときに `True` を返す | `bool` |
| `write(s, /)` | 文字列 `s` をストリームに書き出し、書き出された文字数を返す | `int` |
| `writelines(lines, /)` | バッファに行のリストを書き込む。行区切り文字は追加されないので、書き込む各行の行末に行区切り文字を含ませるのが一般的である | `None` |
| `seekable()` | `io.TextIOWrapper` においては 、このメソッドは常に `True` を返す | `bool` |
| `tell()` | 現在のストリーム位置を返す | `int` |
| `seek(cookie,`<br />` whence=os.SEEK_SET, /)` | ストリームの位置を設定し、新しいストリームの位置を返す。次の引数の組み合わせにより、4 つの操作がサポートされる<br /><br />・`seek(0, os.SEEK_SET)`: ストリームの先頭まで戻る<br /><br />・`seek(cookie, os.SEEK_SET)`: 以前の位置を復元する。`cookie` は、`tell()` によって返される数値である必要がある<br /><br />・`seek(0, os.SEEK_END)`: ストリームの最後まで進む<br /><br />・`seek(0, os.SEEK_CUR)`: 現在のストリーム位置を変更しない | `int` |
| `truncate(size=None, /)` | ストリームのサイズを、指定された `size` バイト（または `size` が指定されていない場合、現在位置）に変更する。現在のストリーム位置は変<br />更されない。このサイズ変更により、現在のファイルサイズを拡大または縮小させることができる。拡大の場合には、新しいファイル領域の<br />内容は空白で埋められる。新しいファイルサイズが返される | `int` |
| `flush()` | `buffer` の `flush()` メソッドを実行した効果を得る（結果的にファイルへの書き込みが行われる） | `None` |
| `close()` | `flush()` を実行し、ストリームを閉じる。`close()` の後にバッファに対する操作（読み込みや書き込みなど）で `ValueError` 例外が発生する | `None` |

標準入力・標準出力・標準エラー出力
----------------------------------

`sys.stdin` および `sys.stdout`、`sys.stderr` はそれぞれ C で書かれた標準入力、標準出力、標準エラー出力のファイルオブジェクトである。`sys.stdin` は読み込み専用であり、`sys.stdout` および `sys.stderr` は書き込み専用である。

In [ ]:
import sys
import io
assert isinstance(sys.stdin, io.TextIOWrapper)
assert not sys.stdout.readable()  # sys.stdout は書き込み専用
sys.stdout.write("test")

test

`sys.stdout.write()` は、組み込みの `print()` 関数で置き換えることができる。 `print()` 関数は、デフォルトで末尾に改行を入れることに注意する。`end` 引数に空文字列を指定して `print(text, end='')` のように呼び出せば、末尾に改行を入れないようになる。

`sys.stdin.read()` は、組み込みの `input()` 関数で置き換えることができる。 `sys.stdin.read()` は <kbd>Ctrl+D</kbd> の入力で入力を終了するが、`input()` 関数は Enter の入力で入力を終了することに注意する。

次のコードは、`sys.stdin`、`sys.stdout`、`sys.stderr` の実装を Python で実現するものである。

``` python
import io
import os
stdin = io.TextIOWrapper(
    io.BufferedReader(
        io.FileIO(0, "r", closefd=False),
    ),
    encoding="locale",
    line_buffering=os.isatty(0),
)
stdout = io.TextIOWrapper(
    io.BufferedWriter(
        io.FileIO(1, "w", closefd=False),
    ),
    encoding="locale",
    line_buffering=os.isatty(1),
)
stderr = io.TextIOWrapper(
    io.BufferedWriter(
        io.FileIO(2, "w", closefd=False),
    ),
    encoding="locale",
    line_buffering=True,
)
```

`io.FileIO()` の引数として、それぞれファイルディスクリプタ 0, 1, 2 を渡している。また、`closefd=False` を指定することで、ストリームを閉じても標準入力などが閉じないようにしている。

`io.TextIOWrapper` の `encoding` 引数は、システムのロケールエンコーディングを指定している。`locale` は Python 3.10 から使用できるようになった。また、`os.isatty(fd)` 関数はファイルディスクリプタ `fd` がオープンされていて、tty（のような）端末である場合（ファイル等にリダイレクトされていない場合）に `True` を返す結果、`line_buffering` 引数が `True` となる（標準エラー出力では常に `True` である）。

Windows では、ロケールエンコーディングがデフォルトでは CP932 であるため、Python の `print()` 関数などで標準出力を使用すると UTF-8 から CP932 への変換が行われる。この時に CP932 にない文字コードがあれば `UnicodeEncodeError` 例外が発生する。次のようにすれば、標準出力のエンコーディングを変更できる。

``` python
sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8', line_buffering=os.isatty(1))
```

あるいは、`sys.stdout` のエンコーディングを変更することもできる。

``` python
sys.stdout.reconfigure(encoding='utf-8')
```

`io.TextIOWrapper` も `io.StringIO` も `io.TextIOBase` を継承し、共通するインターフェースを持つので、標準入力・標準出力を `io.StringIO` と差し替えることができる。

contextlib.redirect_stdout
--------------------------

``` python
contextlib.redirect_stdout(new_target)
contextlib.redirect_stderr(new_target)
```

`contextlib` モジュールが提供するこれらの関数は、それぞれ標準出力 `sys.stdout`、 標準エラー出力 `sys.stderr` を `new_target` にリダイレクトするコンテキストマネージャーである。

リダイレクト先を null デバイス `os.devnull` とすれば、利用する関数やメソッドの中身に手を加えることなく、一時的に出力を無効化することができる:

In [ ]:
import os
from contextlib import redirect_stdout

with redirect_stdout(open(os.devnull, 'w')):
    import this  # The Zen of Python

open() 関数
-----------

テキストストリームおよびバイナリーストリームを作る一番簡単な方法は、組み込み関数 `open()` を利用することである。`open()` の引数は次のとおり:

``` python
open(file, mode='r', buffering=-1, encoding=None, errors=None, newline=None)
```

`file` は、開くファイルを表す path-like オブジェクトである。

`mode` は、Raw I/O に関する指定とバイナリ／テキストに関する指定の 2 種類に分けられる。バイナリ／テキストに関する指定は `'b'`（バイナリ）と `'t'`（テキスト）の 2 つであり、デフォルトは `'t'` となる。

`mode` の指定と、`buffering` の指定の組み合わせにより、`open()` 関数が返すファイルオブジェクトは、次のようになる。

| `mode` | `buffering` | `open()` 関数が返すファイルオブジェクト | 備考 |
|:--:|:--:|:---|:---|
| `'rb'` | 0 | `io.FileIO(os.fspath(file), "r")` | `'wb'`, `'xb'`, `'ab'`, `'r+b'`, `'w+b'`, `'a+b'` も同様 |
| `'rb'` | -1 か 1 より<br />大きい整数 | `io.BufferedReader(io.FileIO(os.fspath(file), "r"), buffer_size)` | `buffer_size` は `buffering == -1` なら `io.DEFAULT_BUFFER_SIZE`<br />そうでないなら `buffering` の値 |
| `'wb'` | -1 か 1 より<br />大きい整数 | `io.BufferedWriter(io.FileIO(os.fspath(file), "w"), buffer_size)` | `'xb'`, `'ab'` も同様。`buffer_size` は上と同じ |
| `'r+b'` | -1 か 1 より<br />大きい整数 | `io.BufferedRandom(io.FileIO(os.fspath(file), "r+"), buffer_size)` | `'w+b'`, `'a+b'` も同様。`buffer_size` は上と同じ |
| `'r'` | 1 | `io.TextIOWrapper(io.BufferedReader(io.FileIO(os.fspath(file), "r")), encoding, errors, newline,`<br />` line_buffering=True)` | |
| `'w'` | 1 | `io.TextIOWrapper(io.BufferedWriter(io.FileIO(os.fspath(file), "w")), encoding, errors, newline,`<br />` line_buffering=True)` | `'x'`, `'a'` も同様 |
| `'r+'` | 1 | `io.TextIOWrapper(io.BufferedRandom(io.FileIO(os.fspath(file), "r+")), encoding, errors, newline,`<br />` line_buffering=True)` | `'w+'`, `'a+'` も同様 |
| `'r'` | -1 | `io.TextIOWrapper(io.BufferedReader(io.FileIO(os.fspath(file), "r")), encoding, errors, newline)` | |
| `'w'` | -1 | `io.TextIOWrapper(io.BufferedWriter(io.FileIO(os.fspath(file), "w")), encoding, errors, newline)` | `'x'`, `'a'` も同様 |
| `'r+'` | -1 | `io.TextIOWrapper(io.BufferedRandom(io.FileIO(os.fspath(file), "r+")), encoding, errors, newline)` | `'w+'`, `'a+'` も同様 |

バイナリモードの場合、`buffering=0` の指定によりバッファリングが無効になる。バイナリモードでは、引数 `encoding` および `errors`、`newline` は使われない。

テキストモードの場合、`buffering=1` の指定により行単位でのバッファリングが有効になる。テキストモードでは、引数 `encoding` および `errors`、`newline` は、そのまま `io.TextIOWrapper` のコンストラクタの引数として渡される。**必ず `encoding='utf-8'` のようにエンコーディングを指定すること**。未指定の場合、OS 環境に依存して決まるので（Windows の日本語環境では、`cp932`）、`UnicodeDecodeError` が発生しやすい。

空のファイルを作成したい場合は、with 文を使って、書き込みモード（`mode='w'` または `mode='x'`）で `open()` を呼び出し、with 文の本体は pass 文のみとすればよい。

``` python
with open(file, 'w'):
    pass
```

`UnicodeDecodeError` などのファイル操作中のエラーを捕捉したい場合は、次のようにする:

``` python
def read_file(file):
    with open(file, encoding="utf-8") as f:
        try:
            content = f.read()
            return content
        except UnicodeDecodeError:
            # 何らかの処理
```

## パスオブジェクトのメソッド

`pathlib.Path` は `open()` 関数の代替となるメソッドを持つ。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `open(mode='r', buffering=- 1, encoding=None,`<br />` errors=None, newline=None)` | パスがファイルを指していた場合、組み込み関数 `open()` のようにパスが指しているファイルを開く | ファイルオ<br />ブジェクト |
| `read_text(encoding=None, errors=None)` | パスがファイルを指していた場合、その内容全体を文字列として返し、ファイルを閉じる | `str` |
| `read_bytes()` | パスがファイルを指していた場合、その内容全体をバイト列として返し、ファイルを閉じる | `bytes` |
| `write_text(data, encoding=None, errors=None,`<br />` newline=None)` | パスがファイルを指していた場合、ファイルをテキストモードで開き、`data` を書き込み、ファイルを閉じる。書き込んだ文字数を返す | `int` |
| `write_bytes(data)` | パスがファイルを指していた場合、ファイルをバイトモードで開き、data を書き込み、ファイルを閉じる。書き込んだバイト数を返す | `int` |

`read_text()`、`read_bytes()`、`write_text()`、`write_bytes()` は必ずファイルを閉じるので、with 文を使う必要がない。`write_text()`、`write_bytes()` は追記ではなく上書きとなり、元の内容は削除されるので注意。

In [ ]:
from pathlib import Path

p = Path("./sample_data/README.md", encoding="utf-8")
with p.open() as f:
    print(f"{f.readline()=}")
text = p.read_text()
print(f"{len(text)=}")

f.readline()='This directory includes a few sample datasets to get you started.\n'
len(text)=930


fileinput
---------

標準ライブラリの `fileinput` モジュールは、複数のファイルを 1 つずつ開いて 1 行ずつ読み込む反復処理に便利な関数を提供する。

``` python
fileinput.input(files=None, inplace=False, backup='', *, mode='r', openhook=None, encoding=None, errors=None)
```

この関数は、シーケンス `files` を処理の対象とする `fileinput.FileInput` インスタンスを作成して返す。

  * `fileinput.FileInput` インスタンスは、シーケンスからファイルを 1 つずつ開いて 1 行ずつ返すイテレーターである。次のファイルを開く前に現在のファイルは閉じられる。
  * `fileinput.FileInput` インスタンスは、コンテキストマネージャーとして使用できる。 with 文のブロックの完了時に、現在開いているファイルは閉じられ、シーケンスが空にされる。

キーワード専用引数 `mode` と `encoding` と `errors` は、ファイルを開く処理の際に `open()` 関数の同名の引数に渡される（ただし `encoding` と `errors` は Python 3.10 で追加された）。このほかの引数については以下のとおり。

| 引数 | 意味 |
|:---|:---|
| `files` | path-like オブジェクトのシーケンスを指定する。省略するか `None` を指定した場合、コマンドライン引数で与えられた全てのファイルが対象となる。ファイル名として `'-'` が与えら<br />れた場合、および、コマンドライン引数でファイルを指定しなかった場合は、標準入力から 1 行ずつ読み込む |
| `inplace` | `True` の場合、入力ファイルはバックアップファイルに移動され、標準出力が入力ファイルに設定される |
| `backup` | 拡張子を指定すると、バックアップファイルの拡張子として利用される。デフォルトの拡張子は `'.bak'` になっている |
| `openhook` | キーワード専用引数。`open()` 関数に代わる関数を指定できる。この関数は、2 つの引数 `filename` と `mode` をとり、ファイルオブジェクトを返す関数でなければならない |

`mode`、`inplace`、`openhook` 引数は、標準入力を読み込んでいる間は無効にされる。

`inplace` と `openhook` を同時に使うことはできないことに注意する。

`fileinput` モジュールは、 `fileinput.input()` 関数によって作られた `fileinput.FileInput` インスタンスをこのモジュールの関数群が利用するグローバルな状態として扱う。以下の関数は、 `fileinput.FileInput` インスタンスのメソッドを利用している。

| 関数 | 機能 | 戻り値 |
|:---|:---|:---|
| `fileinput.filename()` | 現在読み込み中のファイル名を返す。1 行目が読み込まれる前は `None` を返す | `str` &#124; `None` |
| `fileinput.fileno()` | 現在のファイルのファイルディスクリプタを返す。ファイルがオープンされていない場合（最初の行の前、ファイルとファイルの間）は `-1` を返す | `int` |
| `fileinput.lineno()` | 最後に読み込まれた行の、累積した行番号を返す。1 行目が読み込まれる前は `0` を返す | `int` |
| `fileinput.filelineno()` | 現在のファイル中での行番号を返す。1 行目が読み込まれる前は `0` を返す | `int` |
| `fileinput.isfirstline()` | 最後に読み込まれた行がファイルの 1 行目なら `True` 、そうでなければ `False` を返す | `bool` |
| `fileinput.isstdin()` | 最後に読み込まれた行が `sys.stdin` から読み込まれていれば `True`、そうでなければ `False` を返す | `bool` |
| `fileinput.nextfile()` | 現在のファイルを閉じる。次の繰り返しでは（存在すれば）次のファイルの最初の行が読み込まれる。閉じたファイルの読み込まれなかった行は、<br />累積の行数にカウントされない | `None` |
| `fileinput.close()` | 現在開いているファイルを閉じ、シーケンスを空にする。with 文を使用する場合は、この関数を呼び出す必要はない | `None` |

これらの関数があるため、 `fileinput.FileInput` インスタンスは with 文のブロックの中でイテレーターとしてだけ利用される。

In [ ]:
import fileinput
import glob
files = glob.glob("*.txt")
with fileinput.input(files=("sample_data/README.md", "sample_data/california_housing_test.csv"), encoding="utf-8") as fi:
    for line in fi:
        if fileinput.isfirstline():
            print(f"{fileinput.filename()}:")
            print(line.rstrip())
            fileinput.nextfile()
print(f"{fileinput.lineno()=}")

sample_data/README.md:
This directory includes a few sample datasets to get you started.
sample_data/california_housing_test.csv:
"longitude","latitude","housing_median_age","total_rooms","total_bedrooms","population","households","median_income","median_house_value"
fileinput.lineno()=2


``` python
fileinput.hook_compressed(filename, mode, *, encoding=None, errors=None)
```

この関数は、`fileinput.input()` の `openhook` 引数に渡せる。 `gzip` や `bzip2` で圧縮された（拡張子が `'.gz'` や `'.bz2'` の）ファイルを透過的に開く。ファイルの拡張子が `'.gz'` や `'.bz2'` でない場合は、通常通りファイルが開かれる。

使用例: `fi = fileinput.FileInput(openhook=fileinput.hook_compressed, encoding="utf-8")`

chardet
-------

サードパーティ製の [chardet](https://chardet.readthedocs.io/) パッケージは、文字コードを判定する機能を提供する。ライセンスは LGPL-2.1 license。インストール方法は次のとおり。

``` shell
pip install chardet
```

`chardet` では、日本語の主要な文字コードがサポートされる。サポートされる文字コードは公式ドキュメントを参照。

  * [Supported encodings](https://chardet.readthedocs.io/en/latest/supported-encodings.html#supported-encodings)

`chardet.detect()` 関数は、渡されたバイト列から文字コードを調べ、判定結果を辞書として返す。辞書の `'encoding'` キーに判定された文字コードが格納され、 `'confidence'` キーに判定の信頼度が格納される。

In [ ]:
import chardet
rawdata = '''吾輩は猫である。名前はまだ無い。
どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。
'''.encode("cp932")
chardet.detect(rawdata)

{'encoding': 'SHIFT_JIS', 'confidence': 0.99, 'language': 'Japanese'}

文字コード CP932 と Shift_JIS の違いは、文字集合に「NEC拡張文字」「IBM拡張文字」「NEC選定IBM拡張文字」が含まれているかいないかである。バイト列にこれらの文字のコードポイントが含まれていなければ、当然 Shift_JIS と判定されることに注意する。

与えられたバイト列が短すぎると文字コードの予測ができない。

In [ ]:
import chardet
rawdata = '吾輩は猫である。名前はまだ無い。'.encode("cp932")
chardet.detect(rawdata)

{'encoding': None, 'confidence': 0.0, 'language': None}

行数の多いファイルの場合、 `chardet.universaldetector.UniversalDetector` クラスを使用したほうが判定の効率が良い。このクラスは以下の属性とメソッドを持つ。

| 属性 | 意味 |
|:---|:---|
| `done` | `True` ならば、インスタンスは文字コードを予測できる状態である |
| `result` | `'encoding'`, `'confidence'`, `'language'` キーを含む辞書が格納される。この辞書は文字コードを予測した結果を表す |

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `reset()` | インスタンスを初期状態にリセットする | `None` |
| `feed(byte_str)` | 現在のドキュメントの分析を目的としてバイト列 `byte_str` を供給する。このメソッドを呼び出した後に `done` 属性の値をチェックして、さらにインスタンスにデータ<br />を供給する必要があるかどうかを確認するという使い方をする | `None` |
| `close()` | 現在のドキュメントの分析をやめて最終的な予測を立てる。`result` 属性を返す | `dict` |

以下のコードは、ファイルの文字コードを判定する関数 `detect_encoding()` を定義する:

In [ ]:
from chardet.universaldetector import UniversalDetector

detector = UniversalDetector()

def detect_encoding(file):
    detector.reset()
    with open(file, "rb") as f:
        for line in f:
            detector.feed(line)
            if detector.done:
                break
    detector.close()
    return detector.result["encoding"]

detect_encoding("sample_data/README.md")

'ascii'

tqdm
----

サードパーティ製パッケージの [tqdm](https://tqdm.github.io/) は、プログレスバー（進捗状況を表現するためのバー）を表示する機能を提供する。重い処理を for 文で回しているときなど、いつ終わるかわからない繰り返し計算の進捗状況を可視化することで計算時間に検討をつけることができる。ライセンスは MIT License と MPL v 2.0 の組み合わせ。インストール方法は次のとおり。

``` shell
pip install tqdm
```

`tqdm` のインポートには、3 種類の方法がある。

``` python
# テキストベースのプログレスバーを表示する場合
from tqdm import tqdm

# HTML ベースのプログレスバーを表示する場合
from tqdm.notebook import tqdm

# 自動で環境に合わせる場合
from tqdm.auto import tqdm
```

テキストベースのプログレスバーは、ターミナルのようなテキストベースの環境では問題なく動作するが、Colab 上では表示が崩れることがある。

一方、HTML ベースのプログレスバーはスタイリッシュであり、通常、Colab 上では `tqdm.notebook` を使用する。ただし、`tqdm.notebook` を使用したコードはターミナルでは動作しないという問題がある。

`tqdm.auto` を使用すると、コードを実行する環境に応じてプログレスバーの種類が適切に選択されるようになる。環境に依存しないコードを書きたい場合には、`tqdm.auto` を使用するとよい。

基本的な使い方は、`tqdm` クラスのインスタンスを for ループと一緒に使うことである。

In [ ]:
from time import sleep
from tqdm.auto import tqdm

for _ in tqdm(range(1000)):
    sleep(0.001)

  0%|          | 0/1000 [00:00<?, ?it/s]

コンストラクタの第 1 引数 `iterable` にイテラブルを指定する場合、 `tqdm` インスタンスは `iterable` のアイテムを返すイテラブルとなる。

進行状況メッセージは、デフォルトでは次のフォーマットが使用される。

　`'{l_bar}{bar}{r_bar}'`

  * `l_bar`=`'{desc}: {percentage:3.0f}%|'`
  *  `r_bar`=`'| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, ' '{rate_fmt}{postfix}]'`

| プレースホルダ | 意味 |
|:---|:---|
| `{bar}` | プログレスバー |
| `{desc}` | 説明文 |
| `{percentage:3.0f}` | パーセンテージの書式指定 |
| `{n_fmt}` | 現在処理されたアイテムの数 |
| `{total_fmt}` | 全体のアイテムの数 |
| `{elapsed}` | 経過時間（秒） |
| `{remaining}` | 完了までに予想される残り時間（秒） |
| `{rate_fmt}` | 処理速度（アイテム/秒） |
| `{postfix}` | 追加の統計 |

このうち `{desc}: ` と `{postfix}` は、デフォルトでは表示されない。

コンストラクタは、進行状況メッセージをカスタマイズするための引数を多数サポートしている。その主なものは次のとおり。

| 引数 | 意味 | デフォルト |
|:---|:---|:---|
| `desc` | `{desc}` に表示される説明文 | `None` |
| `total` | `{total_fmt}` に表示される全体のアイテムの数。`None` の場合、`len(iterable)` が使用される。`iterable` 引数を `None`<br />（デフォルト）とする場合は、`total` を指定する必要がある。この場合、`tqdm` は整数を返すイテラブルとなる | `None` |
| `leave` | `False` の場合、処理完了後にプログレスバーを非表示にする | `True` |
| `file` | 進行状況メッセージを出力先を指定する。`None` の場合は、`sys.stderr` が使用される | `None` |
| `ncols` | 進行状況メッセージ全体の幅を 0 以上の整数で指定できる。指定した場合、この幅に収まるようにプログレスバーのサイ<br />ズが動的に変更される | `None` |
| `mininterval` | 進行状況表示の最小更新間隔（秒） | `0.1` |
| `disable` | `True` の場合、進行状況メッセージ全体を非表示にする | `False` |
| `unit` | `{rate_fmt}` に使用されるアイテムの単位を指定する | `'it'` |
| `unit_scale` | `True` の場合、`{n_fmt}` と `{total_fmt}` がキロ、メガなどの単位でスケールされる | `False` |
| `bar_format` | 出力の書式設定を指定する。`bar_format` の変更はパフォーマンスに影響を与える可能性がある | `'{l_bar}{bar}{r_bar}'` |
| `initial` | 進行状況の初期値を指定する | `0` |
| `position` | 複数のプログレスバーを使う場合に、プログレスバーの縦位置を指定できる | `None` |
| `postfix` | `{postfix}` に表示される追加の統計。文字列または辞書が使える | `None` |
| `colour` | プログレスバーの色を文字列で指定する（e.g. `'green'`, `'00ff00'`） | `None` |

次のコードは、`desc` 引数、`unit` 引数、`postfix` 引数を指定して進行状況メッセージをカスタマイズする例である。

In [ ]:
from time import sleep
from tqdm.auto import tqdm

for i in tqdm(range(1000), desc="Download", unit="B", unit_scale=True, postfix={"loss": 1.2}):
    sleep(0.001)

Download:   0%|          | 0.00/1.00k [00:00<?, ?B/s, loss=1.2]

コンストラクタの第 1 引数にジェネレーターを渡す場合は、`len()` 関数ではジェネレーターが生成するアイテムの総数を取得できないため、`{total_fmt}` と `{remaining}` が表示されないという問題がある。ジェネレーターが生成するアイテムの総数がわかっているならば、`total` 引数にその数を指定すると `{total_fmt}` と `{remaining}` が表示されるようになる。

In [ ]:
from time import sleep
from tqdm.auto import tqdm

gen1 = (x for x in range(1000))
# total を指定しないと「全体のアイテムの数」「完了までに予想される残り時間」が表示されない
for _ in tqdm(gen1):
    sleep(0.001)
gen2 = (x for x in range(1000))
# total を指定すると「全体のアイテムの数」「完了までに予想される残り時間」が表示されるようになる
for _ in tqdm(gen2, total=1000):
    sleep(0.001)

0it [00:00, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

`tqdm` インスタンスでは、以下のメソッドが使用できる。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `update(n=1)` | 進行状況表示を手動で更新する。`n` で処理したアイテム数を指定する | `True` または `None` |
| `set_description(desc)` | `{desc}` に表示される説明文を更新する | `None` |
| `set_postfix(**kwargs)` | `{postfix}` に表示される追加の統計を更新する | `None` |
| `close()` | クリーンアップする。コンストラクタ引数 `leave` に `False` を渡していた場合、プログレスバーを閉じる | `None` |

`tqdm` はコンテキストマネージャーとして使用できる。`close()` メソッドは、 with ブロックを終了するときに呼び出される。

次は、動的な説明文と追加の統計を表示するコード例:

In [ ]:
from time import sleep
from tqdm.auto import tqdm

with tqdm("abcde") as pbar:
    for i, ch in enumerate(pbar):
        pbar.set_description("[train] Epoch %d" % i)
        pbar.set_postfix(dict(loss=1 - i / 5, acc=i / 10))
        sleep(1.0)

  0%|          | 0/5 [00:00<?, ?it/s]